410921202 資工四 林芷萱 0508 Practice

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 定義triangle_points跟keypoint_names
triangle_points = ["RIGHT_EYE", "LEFT_EYE", "MOUTH_CENTER"]
keypoint_names = ["RIGHT_EYE", "LEFT_EYE", "NOSE_TIP", "MOUTH_CENTER", "RIGHT_EAR_TRAGION", "LEFT_EAR_TRAGION"]

with mp_face_detection.FaceDetection(
    min_detection_confidence=0.5) as face_detection:

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        
        # Run MediaPipe Face Detection.
        results = face_detection.process(image)

        # Draw face detections of each face.
        # print(f'Face detections of {name}:')
        if not results.detections:
            continue
        annotated_image = cv2.cvtColor(image.copy(), cv2.COLOR_RGB2BGR)

        # 初始化存取keypoints的字典
        keypoints = {}

        for detection in results.detections:
            for kp in mp_face_detection.FaceKeyPoint:
                keypoint = mp_face_detection.get_key_point(detection, kp)
                x = int(keypoint.x * frameWidth)
                y = int(keypoint.y * frameHeight)
                keypoints[kp.name] = (x, y)

        # 找triangle_points的點
        triangle = []
        for point in triangle_points:
            if point in keypoints:
                triangle.append(keypoints[point])
                cv2.circle(annotated_image, keypoints[point], 5, (0, 255, 0), -1) # 用綠色的點標記找到的triangle_points

        # 畫出三角形
        if len(triangle) == 3:
            cv2.polylines(annotated_image, [np.array(triangle)], isClosed=True, color=(255, 0, 0), thickness=2)

        # 畫出法線
        if "NOSE_TIP" in keypoints and len(triangle) == 3:
            nose_tip = keypoints["NOSE_TIP"]
            center_of_triangle = (sum([point[0] for point in triangle]) // 3, sum([point[1] for point in triangle]) // 3) # 計算triangle中心點的(x, y)
            cv2.arrowedLine(annotated_image, center_of_triangle, nose_tip, (0, 0, 255), 2)

        cv2.imshow('face detection', annotated_image)
        
        c = cv2.waitKey(1)
        if c == 27:
            break

cap.release()
cv2.destroyAllWindows()
